In [0]:
# Importando as bibliotecas necessárias para o projeto

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [0]:
%python

patients = """
SELECT * FROM dev_db.intermediate.int_patients_symptoms
"""

df_spark = spark.sql(patients)
df_pandas = df_spark.toPandas()


In [0]:
symptom_columns = ['fever', 'cough', 'runny_nose', 'headache', 'body_pain', 'fatigue', 'nausea', 'diarrhea']

for col in symptom_columns:
    df_pandas[col] = df_pandas[col].map({'Yes': 1, 'No': 0})

X = df_pandas[symptom_columns] # Descritivo
y = df_pandas['diagnose'] # O que queremos prever

display(X.head())

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy:.2f}')
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred, zero_division=0))

In [0]:
cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=model.classes_, yticklabels=model.classes_)
plt.title('Matriz de Confusão (Scikit-Learn)')
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.show()

feature_importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x=feature_importances, y=feature_importances.index)
plt.title("Importância de cada Sintoma (Scikit-Learn)")
plt.show()


In [0]:
from sklearn.tree import plot_tree

plt.figure(figsize=(20, 15))

plot_tree(model.estimators_[0]
          , feature_names=X.columns.tolist()
          , class_names=model.classes_.tolist()
          , filled=True
          , rounded=True
          , fontsize=10)

plt.show()